# This notebook is to create atomic strucutres and then LMDB to be the input of ML model (derived from OCP)

In [1]:
import pickle
import json
import lmdb
import pandas as pd

from ase import Atoms, Atom
from ase.visualize import view
from ase.constraints import dict2constraint
from ase.calculators.singlepoint import SinglePointCalculator
from ase.io import write

from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.ext.matproj import MPRester

from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.datasets import SinglePointLmdbDataset

# Retrieve dataset

In [19]:
with open('docs.pkl', 'rb') as file_handle:
    docs = pickle.load(file_handle)
co_set = [doc for doc in docs if doc['adsorbate'] == 'CO']
h_set = [doc for doc in docs if doc['adsorbate'] == 'H']
oh_set = [doc for doc in docs if doc['adsorbate'] == 'OH']
print(len(co_set),len(h_set),len(oh_set))

19101 22857 6512


In [20]:
for a, item in enumerate(co_set):
    if len(item['atoms']['symbol_counts']) == 3:
        try:
            print(item['atoms']['symbol_counts']['Sn'], a)
        except KeyError:
            None

16 764
6 1109
16 1623
16 3086
8 3196
12 4254
6 4430
6 4553
12 5074
12 5517
6 5944
8 7605
8 7833
16 9535
12 11365
16 11689
16 11924
8 11970
12 12015
12 12955
6 13050
12 13420
16 13555
12 14185
8 14314
16 14514
8 14843
16 15633
12 16775
12 17040
8 17120
6 18697


# Multiple facet for one element

In [21]:
m = MPRester('7zd0wqmMOTujHS0Zxp')
data = pd.DataFrame(m.query(criteria = {"elements": {"$in": ['Al']}, "nelements": 1}, 
                            properties = ['task_id', 'pretty_formula',  'spacegroup.crystal_system',
                                       'spacegroup.symbol']))
data

,task_id,pretty_formula,spacegroup.crystal_system,spacegroup.symbol
0,mp-1239196,Al,tetragonal,I4/mmm
1,mp-1183144,Al,hexagonal,P6_3/mmc
2,mp-134,Al,cubic,Fm-3m
3,mp-1245307,Al,triclinic,P1
4,mp-998860,Al,cubic,Im-3m
5,mp-1245067,Al,triclinic,P1
6,mp-1244953,Al,triclinic,P1
7,mp-1245152,Al,triclinic,P1
8,mp-1245129,Al,triclinic,P1


# Choose element and facet

In [22]:
#Defaults
Cu_FCC_id = 'mp-30'
Ni_FCC_id = 'mp-23'
Zn_HCP_id = 'mp-79'
Sn_BCT_id = 'mp-55'
Ag_FCC_id = 'mp-124'
Au_FCC_id = 'mp-81'
Al_FCC_id = 'mp-134'
CO = Atoms('CO', positions=[[0., 0., 0.],[0., 0., 1.2]])
H = Atoms('H', positions=[[0., 0., -0.5]])
OH = Atoms('OH', positions=[[0., 0., 0.],[0.92, 0., 0.32]])
#{111}, {100} and {110} facets, which are usually present in the equilibrium crystal shape of FCC metals
#HCP 0001

# View atoms object from dataset

In [23]:
def make_atoms_from_doc(doc):
    '''
    This is the inversion function for `make_doc_from_atoms`; it takes
    Mongo documents created by that function and turns them back into
    an ase.Atoms object.

    Args:
        doc     Dictionary/json/Mongo document created by the
                `make_doc_from_atoms` function.
    Returns:
        atoms   ase.Atoms object with an ase.SinglePointCalculator attached
    '''
    atoms = Atoms([Atom(atom['symbol'],
                        atom['position'],
                        tag=atom['tag'],
                        momentum=atom['momentum'],
                        magmom=atom['magmom'],
                        charge=atom['charge'])
                   for atom in doc['atoms']['atoms']],
                  cell=doc['atoms']['cell'],
                  pbc=doc['atoms']['pbc'],
                  info=doc['atoms']['info'],
                  constraint=[dict2constraint(constraint_dict)
                              for constraint_dict in doc['atoms']['constraints']])
    results = doc['results']
    calc = SinglePointCalculator(energy=results.get('energy', None),
                                 forces=results.get('forces', None),
                                 stress=results.get('stress', None),
                                 atoms=atoms)
    atoms.set_calculator(calc)
    return atoms

In [25]:
initial_atoms = make_atoms_from_doc(co_set[9401]['initial_configuration'])
view(initial_atoms, viewer='x3d')

# Doping

In [11]:
import random

In [26]:
def doping(atoms,dopant,doping_conc):
    natoms = len(atoms)
    ncatatoms = natoms - 2
    ndopatoms = round(ncatatoms*doping_conc)
    doping_list = random.sample(range(2, natoms), ndopatoms)
    for a in doping_list:
        atoms[a].symbol = dopant
    return 

# create slab (two types of doping function)

In [27]:
# Load functions required
%run -i adsorbate_slab.py

In [28]:
def generate_absorbate_alloy_slab(bulk_id, miller_indices, adsorbate, doping_conc, dopant):
    #bulk
    bulk_atoms = bulk(bulk_id)
    bulk_cn_dict = find_bulk_cn_dict(bulk_atoms)
    #slab
    slab_list = make_slabs_from_bulk_atoms(bulk_atoms, miller_indices)
    #absorbate_slab
    adslab_list = [] ### add dictionary to note down face and dopant later ###

    for slab_atoms in slab_list:
        slab_atoms_tiled, slab_repeat = tile_atoms(atoms=slab_atoms,
                                                    min_x= 4.5,
                                                    min_y= 4.5)
        sites = find_adsorption_sites(slab_atoms_tiled)
        supercell_slab_atoms = slab_atoms_tiled.repeat((2, 2, 1))
        surface_atoms_list = find_surface_atoms_indices(bulk_cn_dict, supercell_slab_atoms)
        adsorbate.euler_rotate(phi=0., theta=0., psi=0.)
        natoms = len(slab_atoms_tiled)
        ndopatoms = round(natoms*doping_conc)
        print(ndopatoms,'doped in', natoms)
        for n in range(100):
            pure_slab_atoms = slab_atoms_tiled.copy()
            doping_list = random.sample(range(natoms), ndopatoms)
            for a in doping_list:
                pure_slab_atoms[a].symbol = dopant

            # Place a uranium atom on the adsorption site and then tag it with
            # a `1`, which is our way of saying that it is an adsorbate
            
            for site in sites:
                adsorption_vector = find_adsorption_vector(bulk_cn_dict, supercell_slab_atoms,
                                                        surface_atoms_list, site)
                # make a copy here so the original adsorbate is not further rotated
                # to align to the adsorption vector at each iteration
                aligned_adsorbate = adsorbate.copy()
                aligned_adsorbate.rotate(np.array([0., 0., 1.]), adsorption_vector) ###################need to be checked#######
                adslab = add_adsorbate_onto_slab(adsorbate=aligned_adsorbate,
                                                slab=pure_slab_atoms,
                                                site=site)
                adslab_list.append(adslab)
        print(len(adslab_list))
    return adslab_list

In [29]:
def generate_absorbate_cat_slab(bulk_id, miller_indices, adsorbate):
    #bulk
    bulk_atoms = bulk(bulk_id)
    bulk_cn_dict = find_bulk_cn_dict(bulk_atoms)
    #slab
    slab_list = make_slabs_from_bulk_atoms(bulk_atoms, miller_indices)
    #absorbate_slab
    adslab_list = [] ### add dictionary to note down face and dopant later ###
    for slab_atoms in slab_list:
            slab_atoms_tiled, slab_repeat = tile_atoms(atoms=slab_atoms,
                                                       min_x= 4.5,
                                                       min_y= 4.5)
            sites = find_adsorption_sites(slab_atoms_tiled)
            supercell_slab_atoms = slab_atoms_tiled.repeat((2, 2, 1))
            surface_atoms_list = find_surface_atoms_indices(bulk_cn_dict, supercell_slab_atoms)
            adsorbate.euler_rotate(phi=0., theta=0., psi=0.)

            # Place a uranium atom on the adsorption site and then tag it with
            # a `1`, which is our way of saying that it is an adsorbate
            
            for site in sites:
                adsorption_vector = find_adsorption_vector(bulk_cn_dict, supercell_slab_atoms,
                                                       surface_atoms_list, site)
                # make a copy here so the original adsorbate is not further rotated
                # to align to the adsorption vector at each iteration
                aligned_adsorbate = adsorbate.copy()
                aligned_adsorbate.rotate(np.array([0., 0., 1.]), adsorption_vector) ###################need to be checked#######
                adslab = add_adsorbate_onto_slab(adsorbate=aligned_adsorbate,
                                             slab=slab_atoms_tiled,
                                             site=site)
                adslab_list.append(adslab)
    return adslab_list

In [45]:
for ad in [CO,OH]:
    for face in [[1,0,0],[1,1,0],[1,1,1]]:
        for n in range(1,4):
            adslab_list = generate_absorbate_cat_slab(Cu_FCC_id, face, ad)
            for atoms in adslab_list:
                doping(atoms,'Zn',n/10)
            write_lmdb(adslab_list,'cu-zn/cu'+str(face[0])+str(face[1])+str(face[2])+str(ad.symbols).lower()+'_zn'+str(n/10)+'.lmdb')
            print('cu-zn/cu'+str(face[0])+str(face[1])+str(face[2])+str(ad.symbols).lower()+'_zn'+str(n/10)+'.lmdb')


cu-zn/cu100co_zn0.1.lmdb
cu-zn/cu100co_zn0.2.lmdb
cu-zn/cu100co_zn0.3.lmdb
cu-zn/cu110co_zn0.1.lmdb
cu-zn/cu110co_zn0.2.lmdb
cu-zn/cu110co_zn0.3.lmdb
cu-zn/cu111co_zn0.1.lmdb
cu-zn/cu111co_zn0.2.lmdb


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000271AC7EFC70>>
Traceback (most recent call last):
  File "d:\SA\envs\ocp-models\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


cu-zn/cu111co_zn0.3.lmdb
cu-zn/cu100oh_zn0.1.lmdb
cu-zn/cu100oh_zn0.2.lmdb


TypeError: remove: path should be string, bytes or os.PathLike, not NoneType

Exception ignored in: 'scipy._lib.messagestream.MessageStream.__dealloc__'
Traceback (most recent call last):
  File "messagestream.pyx", line 91, in scipy._lib.messagestream.MessageStream.close
TypeError: remove: path should be string, bytes or os.PathLike, not NoneType


KeyboardInterrupt: 

In [109]:
for ad in [CO,OH]:
    for face in [[1,0,0],[1,1,0],[1,1,1]]:
        for n in range(1,6):
            adslab_list = generate_absorbate_alloy_slab(Cu_FCC_id, face, ad, n/10, 'Ni')
            write_lmdb(adslab_list,'cu-ni/cu'+str(face[0])+str(face[1])+str(face[2])+str(ad.symbols).lower()+'_ni'+str(n/10)+'.lmdb')
            print('cu-ni/cu'+str(face[0])+str(face[1])+str(face[2])+str(ad.symbols).lower()+'_ni'+str(n/10)+'.lmdb')


2 doped in 16
300
cu-ni/cu100co_ni0.1.lmdb
3 doped in 16
300
cu-ni/cu100co_ni0.2.lmdb
5 doped in 16
300
cu-ni/cu100co_ni0.3.lmdb
6 doped in 16
300
cu-ni/cu100co_ni0.4.lmdb
8 doped in 16
300
cu-ni/cu100co_ni0.5.lmdb
3 doped in 32
400
cu-ni/cu110co_ni0.1.lmdb
6 doped in 32
400
cu-ni/cu110co_ni0.2.lmdb
10 doped in 32
400
cu-ni/cu110co_ni0.3.lmdb
13 doped in 32
400
cu-ni/cu110co_ni0.4.lmdb
16 doped in 32
400
cu-ni/cu110co_ni0.5.lmdb
2 doped in 24
400
cu-ni/cu111co_ni0.1.lmdb
5 doped in 24
400
cu-ni/cu111co_ni0.2.lmdb
7 doped in 24
400
cu-ni/cu111co_ni0.3.lmdb
10 doped in 24
400
cu-ni/cu111co_ni0.4.lmdb
12 doped in 24
400
cu-ni/cu111co_ni0.5.lmdb
2 doped in 16
300
cu-ni/cu100oh_ni0.1.lmdb
3 doped in 16
300
cu-ni/cu100oh_ni0.2.lmdb
5 doped in 16
300
cu-ni/cu100oh_ni0.3.lmdb
6 doped in 16
300
cu-ni/cu100oh_ni0.4.lmdb
8 doped in 16
300
cu-ni/cu100oh_ni0.5.lmdb
3 doped in 32
400
cu-ni/cu110oh_ni0.1.lmdb
6 doped in 32
400
cu-ni/cu110oh_ni0.2.lmdb
10 doped in 32
400
cu-ni/cu110oh_ni0.3.lmdb
13 do

In [10]:
adslab_list = generate_absorbate_cat_slab(Cu_FCC_id, [1,1,1], CO)
doping(adslab_list[0],'Ni',0.5)
view(adslab_list[0], viewer='x3d')

In [11]:
write('Cu-Ni.png', adslab_list[0])

In [115]:
adslab_list[100]

Atoms(symbols='OHCu3NiCu2Ni3Cu2NiCu2Ni3CuNi3CuNiCu', pbc=True, cell=[[5.12123783330632, 0.0, 0.0], [-2.5606189166531585, 4.4351220624652505, 4.703780640173183e-16], [0.0, 0.0, 37.633258629110934]], tags=..., constraint=FixAtoms(indices=[2, 3, 4, 5, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 23]))

In [61]:
dataset = SinglePointLmdbDataset({"src": "lmdbs/co_dev.lmdb"})
len(dataset)

3820

# To LMDB

In [34]:
a2g = AtomsToGraphs(
    max_neigh=50,
    radius=6,
    r_energy=False,
    r_forces=False,
    r_distances=False,
    r_edges=True,
    r_fixed=True,
)

In [40]:
def write_lmdb(atom_list, name):
    db = lmdb.open(
    name,
    #map_size=1099511627776 * 2,
    map_size = 1024 * 1024 * 1024 *2,
    subdir=False,
    meminit=False,
    map_async=True,)
    
    idx = 0
    for item in atom_list:
        # Extract Data object
        data_objects = a2g.convert(item)
        initial_struc = data_objects
    
        initial_struc.sid = idx  # arbitrary unique identifier 
    
        # no neighbor edge case check
        if initial_struc.edge_index.shape[1] == 0:
            print("no neighbors", idx)
            continue
    
        # Write to LMDB
        txn = db.begin(write=True)
        txn.put(f"{idx}".encode("ascii"), pickle.dumps(initial_struc, protocol=-1))
        txn.commit()
        db.sync()
        idx += 1
    db.close()

In [41]:
write_lmdb(adslab_list,"cu/cu100oh.lmdb")

In [42]:
dataset = SinglePointLmdbDataset({"src": "cu/cu111oh.lmdb"})
len(dataset)

AssertionError: No LMDBs found in 'cu\cu111oh.lmdb'